In [6]:
%reload_ext storlets.tools.extensions.ipython
import os
os.environ['OS_AUTH_VERSION'] = '3'
os.environ['OS_AUTH_URL'] = 'http://127.0.0.1/identity/v3'
os.environ['OS_USERNAME'] = 'tester'
os.environ['OS_PASSWORD'] = 'testing'
os.environ['OS_USER_DOMAIN_NAME'] = 'default'
os.environ['OS_PROJECT_DOMAIN_NAME'] = 'default'
os.environ['OS_PROJECT_NAME'] = 'test'

In [7]:
%%storletapp test.TestStorlet

class TestStorlet(object):
    def __init__(self, logger):
        self.logger = logger

    def __call__(self, in_files, out_files, params):
        """
        The function called for storlet invocation
        :param in_files: a list of StorletInputFile
        :param out_files: a list of StorletOutputFile
        :param params: a dict of request parameters
        """
        self.logger.debug('Returning metadata')
        metadata = in_files[0].get_metadata()
        for key in params.keys():
            metadata[key] = params[key]
        out_files[0].set_metadata(metadata)

        self.logger.debug('Start to return object data')
        content = b''
        while True:
            buf = in_files[0].read(16)
            if not buf:
                break
            content += buf
        self.logger.debug('Received %d bytes' % len(content))
        self.logger.debug('Writing back %d bytes' % len(content))
        out_files[0].write(content)
        self.logger.debug('Complete')
        in_files[0].close()
        out_files[0].close()

Upload storlets succeeded /storlet/test.py
Example command `swift download <container> <object> -H X-Run-Storlet:test.py`


In [8]:
myparams = {'color' : 'red'}
%get --storlet test.py --input path:/storlet/test.py  -i myparams -o myresult
print(myresult.headers)
print(myresult.content)

{u'x-object-meta-storlet-language': u'python', u'x-trans-id': u'tx39de631f32cf42d58f407-0058f89455', u'x-object-meta-storlet-main': u'test.TestStorlet', u'transfer-encoding': u'chunked', u'x-object-meta-storlet-interface-version': u'1.0', u'x-object-meta-storlet-object-metadata': u'no', u'x-object-meta-storlet_execution_path': u'/home/swift/test.TestStorlet', u'last-modified': u'Thu, 20 Apr 2017 10:58:29 GMT', u'etag': u'43106d4a48a1f33745e11bd71596d8c4', u'x-timestamp': u'1492685908.66304', u'x-object-meta-color': u'red', u'date': u'Thu, 20 Apr 2017 10:58:29 GMT', u'x-openstack-request-id': u'tx39de631f32cf42d58f407-0058f89455', u'content-type': u'application/octet-stream', u'accept-ranges': u'bytes'}

class TestStorlet(object):
    def __init__(self, logger):
        self.logger = logger

    def __call__(self, in_files, out_files, params):
        """
        The function called for storlet invocation
        :param in_files: a list of StorletInputFile
        :param out_files: a li

In [9]:
%copy --storlet test.py --input path:/storlet/test.py --output path:/log/test.py -i myparams -o myresult
print(myresult.headers)
print(myresult.status)

{u'content-length': u'0', u'x-storlet-generated-from-last-modified': u'Thu, 20 Apr 2017 10:58:29 GMT', u'x-storlet-generated-from-account': u'AUTH_6dbd182dfa9f4ad6ace88992683ee483', u'last-modified': u'Thu, 20 Apr 2017 10:58:32 GMT', u'etag': u'43106d4a48a1f33745e11bd71596d8c4', u'x-trans-id': u'tx8cc66cd8cda643508bee7-0058f89457', u'date': u'Thu, 20 Apr 2017 10:58:31 GMT', u'content-type': u'text/html; charset=UTF-8', u'x-openstack-request-id': u'tx8cc66cd8cda643508bee7-0058f89457', u'x-storlet-generated-from': u'storlet/test.py'}
201


In [10]:
import os
try:
    os.mkdir('/tmp/tmpZl6teg')
except:
    pass
with open('/tmp/tmpZl6teg/storlet_invoke.log', 'w') as f:
    for x in range(10):
        f.write('INFO: sapmle log line')
%put --storlet test.py --input /tmp/tmpZl6teg/storlet_invoke.log --output path:/log/onvoke.log  -i myparams -o myresult
print(myresult.headers)
print(myresult.status)

{u'content-length': u'0', u'last-modified': u'Thu, 20 Apr 2017 10:58:34 GMT', u'etag': u'faccc93089c22cafce9b64e7cc9f2047', u'x-trans-id': u'tx9577116d45ee4e43bc3fd-0058f89458', u'date': u'Thu, 20 Apr 2017 10:58:33 GMT', u'content-type': u'text/html; charset=UTF-8', u'x-openstack-request-id': u'tx9577116d45ee4e43bc3fd-0058f89458'}
201
